In [1]:
import re
import momepy
import numpy as np
import osmnx as ox
import pandas as pd
import networkx as nx
from tqdm import tqdm
from shapely.geometry import LineString, Polygon

In [2]:
# Функция для определения значения REG
def determine_reg(value):
    if isinstance(value, list):
        for item in value:
            if re.match(r'^[МАР]', str(item)):
                return 1
            elif re.match(r'^\d.*[A-Za-zА-Яа-я]', str(item)):
                return 2
        return 3
    elif pd.isna(value):
        return 3
    if re.match(r'^[МАР]', str(value)):
        return 1
    elif re.match(r'^\d.*[A-Za-zА-Яа-я]', str(value)):
        return 2
    else:
        return 3

In [3]:
def custom_map(highway_types):

    maxspeed = {
    'motorway':        110 / 3.6, # Автомагистрали 
    'motorway_link':   110 / 3.6, # Съезды на развязках дорог, на которых действуют те же правила движения, что и на (motorway).
    'primary':         80 / 3.6,  # Автомобильные дороги регионального значения
    'primary_link':    80 / 3.6,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и primary.
    'residential':     60 / 3.6,  # Дороги, которые проходят внутри жилых зон, а также используются для подъезда к ним. 
    'secondary':       70 / 3.6,  # Автомобильные дороги областного значения
    'secondary_link':  70 / 3.6,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и secondary.
    'tertiary':        60 / 3.6,  # Более важные автомобильные дороги среди прочих 
                            # автомобильных дорог местного значения, например
                            # соединяющие районные центры с сёлами, а также несколько сёл между собой.
    'tertiary_link':   60 / 3.6,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и tertiary.
    'trunk':           90 / 3.6,  # Важные дороги, не являющиеся автомагистралями
    'trunk_link':      90 / 3.6,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и trunk.
    'unclassified':    60 / 3.6,  # Остальные автомобильные дороги местного значения, образующие соединительную сеть дорог.
    'living_street':   15 / 3.6   # Дорога с приоритетом у пешеходов
    }

    # Проверяем, является ли highway_types списком.
    if isinstance(highway_types, list):
        # Если значение - список, выбираем тип шоссе с наибольшой максимальной скоростью.
        max_type = max(highway_types, key=lambda x: maxspeed.get(x, np.nan))
        return maxspeed.get(max_type, 40 / 3.6)
    else:
        # Если значение - не список, возвращаем значение или 40, если тип не найден.
        return maxspeed.get(highway_types, 40 / 3.6)

In [4]:
def get_graph_polygon(polygon: Polygon, filter:dict=None, crs:int=3857):
    if not filter:
        filter = "['highway'~'motorway|trunk|primary|secondary|tertiary|unclassified|residential|motorway_link|trunk_link|primary_link|secondary_link|tertiary_link|living_street']"
    graph = ox.graph_from_polygon(polygon, network_type='drive', custom_filter=filter, truncate_by_edge=True)


    nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)
    edges['reg'] = edges['ref'].apply(determine_reg)
    nodes = nodes.to_crs(epsg=crs)
    edges = edges.to_crs(epsg=crs)
    edges['maxspeed'] = edges['highway'].apply(lambda x: custom_map(x))

    nodes_coord = nodes.geometry.apply(
        lambda p: {"x": p.coords[0][0], "y": p.coords[0][1]}
    ).to_dict()

    graph_type = 'car'
    edges = edges[["highway", "node_start", "node_end", "geometry", 'maxspeed', 'reg']]
    edges["type"] = graph_type
    edges["geometry"] = edges["geometry"].apply(
        lambda x: LineString([tuple(round(c, 6) for c in n) for n in x.coords] if x else None)
    )

    travel_type = "walk" if graph_type == "walk" else "car"

    G = nx.MultiDiGraph()
    for _, edge in tqdm(edges.iterrows(), total=len(edges), desc=f"Collecting {graph_type} graph", leave=False):
        p1 = int(edge.node_start)
        p2 = int(edge.node_end)
        geom = (LineString(([(nodes_coord[p1]["x"], nodes_coord[p1]["y"]),
                            (nodes_coord[p2]["x"], nodes_coord[p2]["y"]),])) if not edge.geometry else edge.geometry)
        length=geom.length
        time_sec = length/edge.maxspeed
        G.add_edge(
            p1,
            p2,
            length_meter=length,
            geometry=str(geom),
            type=travel_type,
            time_sec= time_sec,
            highway=edge.highway,
            maxspeed=edge.maxspeed,
            reg=edge.reg
        )
    nx.set_node_attributes(G, nodes_coord)
    G.graph["crs"] = "epsg:" + str(crs)
    G.graph["graph_type"] = travel_type + " graph"

    return G

In [5]:
def get_graph_polygon(polygon: Polygon, filter: dict = None, crs: int = 3857):
    if not filter:
        filter = "['highway'~'motorway|trunk|primary|secondary|tertiary|unclassified|residential|motorway_link|trunk_link|primary_link|secondary_link|tertiary_link|living_street']"
    
    graph = ox.graph_from_polygon(polygon, network_type='drive', custom_filter=filter, truncate_by_edge=True)
    nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)
    
    edges['reg'] = edges['ref'].apply(determine_reg)
    nodes = nodes.to_crs(epsg=crs)
    edges = edges.to_crs(epsg=crs)
    edges['maxspeed'] = edges['highway'].apply(lambda x: custom_map(x))
    edges['time_sec'] = edges['length'] / (edges['maxspeed'])
    
    nodes_coord = nodes.geometry.apply(lambda p: {"x": p.coords[0][0], "y": p.coords[0][1]}).to_dict()
    
    graph_type = 'car'
    edges = edges[["highway", "node_start", "node_end", "geometry", 'maxspeed', 'time_sec', 'reg']]
    edges["type"] = graph_type
    edges["geometry"] = edges["geometry"].apply(lambda x: LineString([tuple(round(c, 6) for c in n) for n in x.coords] if x else None))

    travel_type = "walk" if graph_type == "walk" else "car"
    
    G = nx.MultiDiGraph()
    
    for _, edge in tqdm(edges.iterrows(), total=len(edges), desc=f"Collecting {graph_type} graph", leave=False):
        p1 = int(edge.node_start)
        p2 = int(edge.node_end)
        
        if pd.isna(edge.geometry):
            geom = LineString([(nodes_coord[p1]["x"], nodes_coord[p1]["y"]),
                               (nodes_coord[p2]["x"], nodes_coord[p2]["y"])])
        else:
            geom = edge.geometry
        
        length = geom.length
        
        G.add_edge(
            p1,
            p2,
            length_meters=length,
            geometry=str(geom),
            type=travel_type,
            time_sec=edge.time_sec,
            highway=edge.highway,
            maxspeed=edge.maxspeed,
            reg=edge.reg
        )
    
    nx.set_node_attributes(G, nodes_coord)
    G.graph["crs"] = "epsg:" + str(crs)
    G.graph["graph_type"] = travel_type + " graph"
    
    return G


In [6]:
city = ox.geocode_to_gdf('R1549169', by_osmid=True)
graph = get_graph_polygon(city.unary_union, crs=32636)

C:\Users\local\AppData\Local\Temp\ipykernel_17556\4105506105.py:6: UserWarning: Approach is not set. Defaulting to 'primal'.
  nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)


In [7]:
list(graph.nodes(data=True))[0]
list(graph.edges(data=True))[0]

(0,
 6065,
 {'length_meters': 2044.1436972875738,
  'geometry': 'LINESTRING (1978714.074342 6306698.26278, 1978525.446409 6306479.525903, 1978241.347264 6306150.589832, 1978191.500189 6306094.874168, 1978140.605008 6306040.149655, 1978036.051665 6305932.500188, 1977998.896835 6305895.407747, 1977563.548686 6305475.364534, 1977301.089876 6305223.644212)',
  'type': 'car',
  'time_sec': 79.42488000000002,
  'highway': 'trunk',
  'maxspeed': 25.0,
  'reg': 1})